In [ ]:
# -*- coding: utf-8 -*-
import pymysql
pymysql.install_as_MySQLdb()
import  logging
logger = logging.getLogger(__name__)

'''
一个MySQLdb的包装
obj = CMySqlAgent()
listdata = obj.exec_sql_str_with_ret()
num = obj.exec_sql_str()
'''


class CMySqlAgent(object):
    def __init__(self, _host, _port, _user, _passwd, _db):
        self.host = _host
        self.port = _port
        self.user = _user
        self.passwd = _passwd
        self.db = _db
        self.cur = None
        self.conn = pymysql.connect(host=self.host, port=self.port,
                                    user=self.user, passwd=self.passwd, db=self.db, use_unicode=True,charset='utf8')

    def exec_sql_str(self, sql):
        ret = 0
        self.reconnect()
        try:
            self.cur = self.conn.cursor()
            self.cur.execute(sql)
            ret = self.cur.rowcount
            self.conn.commit()
        except Exception as err:
            logger.error("errmark:%s" % err)
            self.conn.rollback()
        self.cur.close()
        return ret

    def exec_sql_str_with_ret(self, sql):
        ret = []
        self.reconnect()
        try:
            self.cur = self.conn.cursor()
            self.cur.execute(sql)
            data = self.cur.fetchall()
            self.conn.commit()
            fields = [k[0] for k in self.cur.description]
            ret = [dict(zip(fields, row)) for row in data]
        except Exception as err:
            logger.error("errmark:%s" % err)
            self.conn.rollback()
        self.cur.close()
        return ret

    def reconnect(self):
        try:
            self.conn.ping(True)
        except Exception as err:
            logger.error("errmark:%s" % err)
            pass


    def destroy(self):
        if self.conn is not None:
            self.conn.close()

    def __del__(self):
        self.destroy()

if __name__ == '__main__':
    agent = CMySqlAgent('10.225.30.114', 3340, 'root', 'xxx', 'db')
    ret = agent.exec_sql_str_with_ret("select * from t_db limit 5")
    print( ret )











